# Dependências

In [ ]:
#!pip install numba==0.49.1
!pip install numba==0.56.4
#!pip install numba
#!pip uninstall numba -y
#!pip install llvmlite

In [ ]:
#!pip install llvmlite --ignore-installed

In [ ]:
#!pip install gensim==3.6.0
#!pip install binary gensim==3.6.0

In [ ]:
#!pip3 uninstall numpy -y

In [ ]:
#!pip install numpy

# Bibliotecas

In [55]:
import pandas as pd
import numpy as np
import psycopg2 as postgres
import psycopg2.extras
import math
import csv
import random

from gensim.models import Word2Vec
from tqdm import tqdm
#from sshtunnel import SSHTunnelForwarder
from tqdm import tqdm

import numba
from numba import njit
#from numba import cuda

# Funções

## Utilidades

In [35]:
def executeQuery(conn, sql):
    record = None
    try:
        #print(sql)
        cur = conn.cursor(cursor_factory = psycopg2.extras.RealDictCursor)
        cur.execute(sql)
        record = cur.fetchall()
        cur.close()
    except postgres.Error as e:
        print(e)
        cur.execute("ROLLBACK")
        cur.close()
    return record

In [36]:
def executeInsert(conn, sql):
    sucess = False
            
    try:
        #print(sql)
        cur = conn.cursor()
        cur.execute(sql)
        conn.commit()
        cur.close()
        sucess = True
    except postgres.Error as e:
        print(e)
        cur.execute("ROLLBACK")
        cur.close()

    return sucess

In [37]:
#Recebe um id e retorna as categorias e checkin do lugar
def getPOIInformation(conn, business_id):
    
    sql = """
        SELECT id, checkin_count, name, level FROM pois_information WHERE id  = \'"""+str(business_id)+ """\'
    ;"""

    result = executeQuery(conn, sql)

    return result

In [38]:
#Recebe um id e retorna as categorias e checkin do lugar
def getZoningInformation(conn, object_id):
    
    sql = """
        SELECT id FROM zoning_1618_pois WHERE objectid  = \'"""+str(object_id)+ """\'
    ;"""

    result = executeQuery(conn, sql)

    return result

In [39]:
#Recebe um id e retorna as categorias e checkin do lugar
def getZones(conn, object_id):
    
    sql = """
        SELECT objectid FROM zoning_1618_pois
    ;"""

    result = executeQuery(conn, sql)

    return result

In [40]:
#Encontra todos os pontos do bin centrado em um POI trazendo suas informações (categorias e checkin)
def getBinPOIsInformation(conn, business_id, bin_number):

    result = None

    sql = """
        SELECT fk_poi_id_context as id, name, level, checkin_count, distance_m 
        FROM bins_pois_information 
        WHERE fk_poi_id_center = \'"""+str(business_id)+"""\' AND fk_bin_number = """+str(bin_number)+""" order by distance_m asc;"""

    result = executeQuery(conn, sql)

    return result

In [41]:
def getPOIsDistance(conn, center_poi_id, context_poi_id):
    
    result = None

    sql = """
        SELECT distance_m
        FROM has_context_poi 
        WHERE fk_poi_id_center = \'"""+str(center_poi_id)+"""\' AND fk_poi_id_context = """+str(context_poi_id)+""";"""

    result = executeQuery(conn, sql)

    return result

In [42]:
#Recebe um id e retorna as categorias e checkin do lugar
def getAllPOIInformation(conn):
    
    sql = """
        SELECT id as business_id, latitude, longitude, categories, checkin_count FROM poi;"""

    result = executeQuery(conn, sql)

    return result

In [43]:
#LENTO
def calculate_L(POI_path, df):
    dst = 0
    segment = []
    for poi_id in POI_path:
        #print(poi_id)
        segment.append(poi_id)
        if (len(segment) > 2):
            segment.pop(0)

        if (len(segment) == 2):

            p_idx = POI_path[-2]
            p_idx = df.query("business_id=='"+str(p_idx)+"'").index[0]
            n_idx = POI_path[-1]
            n_idx = df.query("business_id=='"+str(n_idx)+"'").index[0]
        
            dst += distances_matrix[p_idx][n_idx]
            
    return dst

In [44]:
#Meio LENTO
def calculate_L_(POI_path, df):
    dst = 0
    
    for i in range(len(POI_path)-1):

        p_idx = POI_path[i]
        p_idx = df.query("business_id=='"+str(p_idx)+"'").index[0]
        n_idx = POI_path[i+1]
        n_idx = df.query("business_id=='"+str(n_idx)+"'").index[0]

        dst += distances_matrix[p_idx][n_idx]
            
    return dst

In [45]:
#Mais Rápido
def calculate_Li_(POI_path, df, i, L):

    p_idx = POI_path[i-1]
    p_idx = df.query("business_id=='"+str(p_idx)+"'").index[0]

    c_idx = POI_path[i]
    c_idx = df.query("business_id=='"+str(c_idx)+"'").index[0]

    n_idx = POI_path[i+1]
    n_idx = df.query("business_id=='"+str(n_idx)+"'").index[0]

    #Retirando antigo segmento
    segment_distance = L - distances_matrix[p_idx][n_idx]
    
    #Tamanho do novo segmento
    dst = segment_distance + distances_matrix[p_idx][c_idx] + distances_matrix[c_idx][n_idx]
            
    return dst

In [46]:
#Essa versão não utilizada a otimização do Numba
def generate_segment(W, L, POI_path):
    
    final_path = [pt for pt in POI_path]

    for wp in W:

        shortest_path = np.inf
        optimal_list = None

        #Testar WP em todas as posições entre 1 e len(list)-2
        for j in range(1, len(final_path)):
            
            local_list = [fp for fp in final_path]
            
            #Testar WP em todas as posições entre 1 e len(list)-2
            local_list.insert(j, wp)
            

            p_idx = local_list[j-1] 
            c_idx = local_list[j]
            n_idx = local_list[j+1]

            #Retirando antigo segmento
            segment_distance = L - distances_matrix[p_idx][n_idx]

            #Tamanho do novo segmento
            dst = segment_distance + distances_matrix[p_idx][c_idx] + distances_matrix[c_idx][n_idx]

            if (dst < shortest_path):
                shortest_path = dst
                optimal_list = [ll for ll in local_list] 
            
        final_path = [ol for ol in optimal_list] 
        L = shortest_path
    
    return final_path

In [47]:
# function optimized to run on gpu 
#@jit(target_backend='cuda')
#@numba.njit
#from numba.typed import List

@njit
#@cuda.jit
def generate_segment_numba(W, L, POI_path):
    
    final_path = [pt for pt in POI_path]

    for wp in W:

        shortest_path = np.inf
        optimal_list = None

        #Testar WP em todas as posições entre 1 e len(list)-2
        for j in range(1, len(final_path)):
            
            local_list = [fp for fp in final_path]
            
            #Testar WP em todas as posições entre 1 e len(list)-2
            local_list.insert(j, wp)
            

            p_idx = local_list[j-1] 
            c_idx = local_list[j]
            n_idx = local_list[j+1]

            #Retirando antigo segmento
            segment_distance = L - distances_matrix[p_idx][n_idx]

            #Tamanho do novo segmento
            dst = segment_distance + distances_matrix[p_idx][c_idx] + distances_matrix[c_idx][n_idx]

            if (dst < shortest_path):
                shortest_path = dst
                optimal_list = [ll for ll in local_list] 
            
        final_path = [ol for ol in optimal_list] 
        L = shortest_path
    
    return final_path

In [48]:
from timeit import default_timer as timer    

# normal function to run on cpu 
def func(a):                                 
    for i in range(100000): 
        a[i]+= 1      

# function optimized to run on gpu  
#@jit(nopython=True)
#@cuda.jit
#(target ="cuda")    
@njit
def func2(a): 
    for j in range(100000):
        for i in range(100000): 
            a[i]+= 1
#if __name__=="__main__": 

n = 100000                           
a = np.ones(n, dtype = np.float64) 
#b = np.ones(n, dtype = np.float32) 

start = timer() 
func(a) 
print("without GPU:", timer()-start)     

start = timer() 
func2(a) 
print("with GPU:", timer()-start) 

without GPU: 0.06260432198178023
with GPU: 4.991320410976186


# Document Corpus

In [56]:
df_NYC = pd.read_csv("../dataset/NYC/NYC_train.csv")
mvp = df_NYC["POI_id"].value_counts()

In [57]:
mvp

42911d00f964a520f5231fe3    643
42829c80f964a5206a221fe3    519
4a737bf8f964a52091dc1fe3    351
4840fe6bf964a52030501fe3    339
40b68100f964a5207d001fe3    302
                           ... 
4a624a38f964a520aac31fe3      1
4f0471fc4901a1400b35a1e7      1
4f595545e4b081a30ea1f6b7      1
501a82d6e4b03c99d14aa5b8      1
503bee84e4b0e79b22202bf1      1
Name: POI_id, Length: 4980, dtype: int64

In [33]:
pois_idx

[2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2]

In [59]:
# TESTE LOCAL

object_id = "4bf58dd8d48988d103941735"	
df = df_NYC

# print('processing POI:', poi_id)
POI_path_bi = []
POI_path_fl = []

# print('processing:', object_id)

# Obtendo informações de categorias e checkin do poi central
# [business_id, checkin, category]
# poi_information = getZoningInformation(connection, object_id)
poi_information = poi_df[poi_df['POI_catid'] == object_id]
poi_information.head()

# Executa o processo se há dados
# Um caminho preciso do no mínimo dois POIs
if len(poi_information) >= 2:

    # Restringindo as possibilidades de tipos
    columns = list(dict(poi_information.iloc[0]).keys())
    poi_information = pd.DataFrame(poi_information, columns=columns)

    # Shortest Path construction
    pois_idx = []
    for i, cp_idx in poi_information.iterrows():
        pois_idx.append(df.query("POI_id=='" + str(cp_idx['POI_id']) + "'").index[0])

    # Encontrar Ps e Pe
    max_distances = []
    for poi_idx_01 in pois_idx:
        for poi_idx_02 in pois_idx:
            if poi_idx_01 != poi_idx_02:
                max_distances.append((poi_idx_01, poi_idx_02, distances_matrix[poi_idx_01][poi_idx_02]))

    furthest_pair = max_distances[0]
    for md in max_distances:
        if md[2] > furthest_pair[2]:
            furthest_pair = md

    POI_path = [furthest_pair[0], furthest_pair[1]]
    L = furthest_pair[2]

    # A is all POIs
    W = set(pois_idx).difference(set(POI_path))

    # Ajuste das variáveis para a função generate_segment
    W = list(W)

    # Adicionar o POI de A que tem o menor tamanho de caminho em L e atualizar W
    if len(W) > 0:

        # Não otimizado
        # POI_path_fl = generate_segment(W, L, POI_path)

        # Otimizado pelo numba
        POI_path_fl = generate_segment_numba(W, L, POI_path)
    else:

        POI_path_fl = POI_path.copy()

    POI_path_bi = [df.iloc[pp]['POI_id'] for pp in POI_path_fl]

In [60]:
#RESULTADO
POI_path_bi

['4c65885618b676b006b0ab0e',
 '4f677137e4b06cb9fdedbe32',
 '4c0bc6377e3fc9285abbf582',
 '4e2ce53dd22d3f83c89eb007',
 '4f63d1cae4b02cbb7e75b3ce',
 '4f44fd5ce4b0c38dc8de1285',
 '4eb27667490126a5703b7af8',
 '4dc83583d22dad7970c7a884',
 '4cdf4390df986ea8d331e116',
 '4fb18686e4b0119ccdadc772',
 '4c2a9a6ab34ad13a0196e8ce',
 '4beb073ea9900f47e2461740',
 '4e359a41814d2eb947337c4e',
 '4e4b1390aeb72792c336098a',
 '4d4fea287ee1a35d65138034',
 '4ee92eff2c5bbde217087aa0',
 '4d5895ca16146a3169b15dc1',
 '4df2c095d22d8c5470dba3ae',
 '4e540e4018381367473717c8',
 '4de85c3cd1648c979631c77c',
 '4f66b7c3e4b08ac7271179ee',
 '4f693f8de4b074b7daf6c0f2',
 '4e34cc321f6e9e21ad484047',
 '4cc24bac4fcfbfb79547bd24',
 '4f2bc3e4e4b05d297c0b55c0',
 '4e63bfa345ddda352a235a77',
 '4c6c52dbf2846dcb299b62e9',
 '4c6fc1379375a0930fc70437',
 '4bca2915cc8cd13aef1bbdcf',
 '4fa7629de4b028d55b6dc3b7',
 '4ce86f18f1c6236afae35ff0',
 '4e532e13c65bea7986d08e3b',
 '4dd892fcfa76ad96d1425e3f',
 '4f5455b3e4b050039fa910c4',
 '4f124a38e4b0

In [61]:
#TIPO DE DADO DO POI_INFORMATION
poi_information

,user_id,POI_id,POI_catid,POI_catid_code,POI_catname,latitude,longitude,timezone,UTC_time,local_time,day_of_week,norm_in_day_time,trajectory_id,norm_day_shift,norm_relative_time
2,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,6,Home (private),40.716162,-73.883070,-240,2012-04-03 18:02:24+00:00,2012-04-03 14:02:24+00:00,1,0.583333,69_1,0,0.583333
3,428,4ce1863bc4f6a35d8bd2db6c,4bf58dd8d48988d103941735,6,Home (private),40.619151,-74.035888,-240,2012-04-03 18:06:18+00:00,2012-04-03 14:06:18+00:00,1,0.583333,428_1,0,0.583333
25,768,4e79392ce4cdb158f1cf4b75,4bf58dd8d48988d103941735,6,Home (private),40.802042,-74.000633,-240,2012-04-03 18:34:00+00:00,2012-04-03 14:34:00+00:00,1,0.604167,768_1,0,0.604167
35,1016,4e5dbf1e1f6e3022a71c242b,4bf58dd8d48988d103941735,6,Home (private),40.912205,-74.012385,-240,2012-04-03 18:55:51+00:00,2012-04-03 14:55:51+00:00,1,0.604167,1016_1,0,0.604167
52,445,4cd544d894848cfa6a0de5b1,4bf58dd8d48988d103941735,6,Home (private),40.824777,-73.883070,-240,2012-04-03 19:20:46+00:00,2012-04-03 15:20:46+00:00,1,0.625000,445_1,0,0.625000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39941,1050,4e51dcc76284416669b03aec,4bf58dd8d48988d103941735,6,Home (private),40.701566,-73.988880,-240,2012-05-18 23:02:37+00:00,2012-05-18 19:02:37+00:00,4,0.791667,1050_4,0,0.791667
39956,173,4e19dabd6284ea724705f1c2,4bf58dd8d48988d103941735,6,Home (private),40.883936,-73.883070,-240,2012-05-18 23:16:39+00:00,2012-05-18 19:16:39+00:00,4,0.791667,173_11,0,0.791667
39971,648,4c19afcb6a21c9b60405cb97,4bf58dd8d48988d103941735,6,Home (private),40.699546,-73.836025,-240,2012-05-18 23:28:34+00:00,2012-05-18 19:28:34+00:00,4,0.791667,648_16,0,0.791667
39976,207,4e15ee97814dbda3c8f946a8,4bf58dd8d48988d103941735,6,Home (private),40.844945,-74.094631,-240,2012-05-18 23:35:22+00:00,2012-05-18 19:35:22+00:00,4,0.812500,207_12,4,4.812500


In [63]:
#DATAFRAME DE POI
poi_df = poi_df[['POI_id', 'latitude', 'longitude']]
poi_df.head()

,POI_id,latitude,longitude
0,49bbd6c0f964a520f4531fe3,40.719810,-74.002581
1,4a43c0aef964a520c6a61fe3,40.606800,-74.044170
2,4c5cc7b485a1e21e00d35711,40.716162,-73.883070
3,4ce1863bc4f6a35d8bd2db6c,40.619151,-74.035888
4,4c0ab56f7e3fc9288c1df482,40.741191,-73.989663


# Carregando dados de POIs

In [ ]:
with SSHTunnelForwarder(
    ('150.165.15.171', 23456),
    #ssh_private_key="</path/to/private/ssh/key>",
    ### in my case, I used a password instead of a private key
    ssh_username="salatiel",
    ssh_password="Salatiel@Lacina123",
    remote_bind_address=('localhost', 5432)) as server:

        server.start()
        #print("server connected")

        params = {'database': 'boston',
               'user': 'salatiel',
               'password': 'root',
               'host': 'localhost',
               'port': server.local_bind_port
        }

        connection = psycopg2.connect(**params)

        #Buscar informações dos POIs
        poi_df = getAllPOIInformation(connection)
        if (len(poi_df) > 0):
            tags = list(dict(poi_df[0]).keys())
            poi_df = pd.DataFrame(poi_df, columns = tags)

poi_df = poi_df.dropna()
poi_df['latitude'] = poi_df['latitude'].astype(float)
poi_df['longitude'] = poi_df['longitude'].astype(float)
poi_df.head()

In [50]:
from scipy.spatial.distance import cdist
distances_matrix = cdist(df[['latitude', 'longitude']].values, df[['latitude', 'longitude']].values)
distances_matrix[0]

array([0.        , 0.12042037, 0.11956666, ..., 0.05230733, 0.04021922,
       0.06065681])

# Carregando dados de Zonas

In [ ]:
#Carregando ZONAS
with SSHTunnelForwarder(
    ('150.165.15.171', 23456),
    #ssh_private_key="</path/to/private/ssh/key>",
    ### in my case, I used a password instead of a private key
    ssh_username="salatiel",
    ssh_password="Salatiel@Lacina123",
    remote_bind_address=('localhost', 5432)) as server:

        server.start()
        #print("server connected")

        params = {'database': 'boston',
               'user': 'salatiel',
               'password': 'root',
               'host': 'localhost',
               'port': server.local_bind_port
        }

        connection = psycopg2.connect(**params)

        #Buscar informações dos POIs
        query = "SELECT distinct(objectid) FROM zoning_1618_pois"
        result = executeQuery(connection, query)
        
        final_zoning_gdf = pd.DataFrame(result)

In [ ]:
final_zoning_gdf.head()

In [ ]:
obj_id_list = list(final_zoning_gdf['objectid'])
obj_id_list[:5]

In [ ]:
results = []
for obj_id in tqdm(obj_id_list):
    results.append(Shortest_Path_FT(obj_id, poi_df))

In [ ]:
Shortest_Path_FT("12269", poi_df)

In [ ]:
#file_name = './Atlanta/Shortest Path/atlanta-shortest_path-zoning-FT_gpu.csv'
file_name = './Boston/Shortest Path/boston-shortest_path-zoning-FT_gpu.csv'

csv_file_02 = open(file_name, "w", newline='')
writer_path = csv.writer(csv_file_02, delimiter=',')
for result in results:
    if(len(result) > 0):
        writer_path.writerow(result)


## Execução em Paralelo

In [ ]:
import multiprocessing as mp

from timeit import default_timer as timer
start = timer() 

print("Number of processors: ", mp.cpu_count())

# Step 1: Init multiprocessing.Pool()
#pool = mp.Pool(int(mp.cpu_count()))
pool = mp.Pool(int(2))

# Step 2: `pool.apply` the `howmany_within_range()`
items = [(object_id, poi_df) for object_id in obj_id_list]

results = pool.starmap(Shortest_Path_FT, items)

#Step 2.1 Writing results

#file_name = './geographic/SHORTEST PATH/austin-shortest_path-zoning-FT_gpu.csv'
file_name = './Atlanta/Shortest Path/atlanta-shortest_path-zoning-FT_gpu.csv'

csv_file_02 = open(file_name, "w", newline='')
writer_path = csv.writer(csv_file_02, delimiter=',')
for result in results:
    writer_path.writerow(result)

csv_file_02.close()

# Step 3: Don't forget to close
pool.close()
    
#print("without GPU:", timer()-start)
print("with GPU:", timer()-start) 

In [ ]:
print(len(results))

In [ ]:
#file_name = './geographic/SHORTEST PATH/austin-sl-corpus-' + str(bin_number) + '-c_.csv'
file_name = './geographic/SHORTEST PATH/austin-shortest_path-' + str(bin_number) + '-CF-c_.csv'
#file_name = './geographic/SHORTEST PATH/austin-shortest_path-' + str(bin_number) + '-FT-c.csv'

csv_file_02 = open(file_name, "w", newline='')
writer_path = csv.writer(csv_file_02, delimiter=',')
for result in results:
    writer_path.writerow(result)

In [ ]:
print('ok')